In [15]:
import time
import numpy as np
import torch.nn.functional as F
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
import torch

In [16]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
random_seed = 1
learning_rate = 0.1
num_epochs = 10
batch_size = 64

num_features = 784
num_classes = 10
num_linear1 = 128
num_linear2 = 256

train_dataset = datasets.MNIST(root='data', train = True, transform=transforms.ToTensor(),download = True)
test_dataset = datasets.MNIST(root='data',train=False,transform=transforms.ToTensor())
train_loader = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_loader = DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)

for images, labels in train_loader:
  print('Image batch dimensions:', images.shape)
  print('Image label dimensions:', labels.shape)
  break

Image batch dimensions: torch.Size([64, 1, 28, 28])
Image label dimensions: torch.Size([64])


In [17]:
class MLP(torch.nn.Module):
  def __init__(self, num_features ,num_classes):
    super(MLP,self).__init__()

    self.linear1 = torch.nn.Linear(num_features,num_linear1)
    self.linear1.weight.detach().normal_(0.0,0.1)
    self.linear1.bias.detach().zero_()
    self.linear1_bn = torch.nn.BatchNorm1d(num_linear1)

    self.linear2 = torch.nn.Linear(num_linear1,num_linear2)
    self.linear2.weight.detach().normal_(0.0,0.1)
    self.linear2.bias.detach().zero_()
    self.linear2_bn = torch.nn.BatchNorm1d(num_linear2)

    self.linear_out = torch.nn.Linear(num_linear2,num_classes)
    self.linear_out.weight.detach().normal_(0.0,0.1)
    self.linear_out.bias.detach().zero_()

  def forward (self,x):
    out = self.linear1(x)
    out = self.linear1_bn(out)
    out = F.relu(out)
    out = self.linear2(out)
    out = self.linear2_bn(out)
    out = F.relu(out)

    logits = self.linear_out(out)
    probas = F.softmax(logits, dim=1)
    return logits, probas

torch.manual_seed(random_seed)
model = MLP(num_features=num_features,num_classes=num_classes)
model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

In [19]:
def compute_accuracy(net,data_loader):
  net.eval()
  correct_pred, num_examples = 0,0
  with torch.no_grad():
    for features,targets in data_loader:
      features = features.view(-1,28*28).to(device)
      targets = targets.to(device)
      logits,probas = net(features)
      _,predicted_labels = torch.max(probas,1)
      num_examples += targets.size(0)
      correct_pred += (predicted_labels ==targets).sum()
      return correct_pred.float()/num_examples*100

start_time = time.time()
for epoch in range(num_epochs):
  model.train()
  for batch_idx, (features,targets) in enumerate(train_loader):
    features = features.view(-1,28*28).to(device)
    targets =targets.to(device)

    logits, probas = model(features)
    cost = F.cross_entropy(logits,targets)
    optimizer.zero_grad()
    cost.backward()

    optimizer.step()
    if not batch_idx %50:
      print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f'
                   %(epoch+1, num_epochs, batch_idx,
                     len(train_loader), cost))

    print('Epoch: %03d/%03d training accuracy: %.2f%%' % (
          epoch+1, num_epochs,
          compute_accuracy(model, train_loader)))

    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))

print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))

串流輸出內容已截斷至最後 5000 行。
Time elapsed: 2.26 min
Epoch: 008/010 training accuracy: 100.00%
Time elapsed: 2.26 min
Epoch: 008/010 training accuracy: 96.88%
Time elapsed: 2.26 min
Epoch: 008/010 training accuracy: 100.00%
Time elapsed: 2.26 min
Epoch: 008/010 training accuracy: 95.31%
Time elapsed: 2.26 min
Epoch: 008/010 training accuracy: 100.00%
Time elapsed: 2.26 min
Epoch: 008/010 training accuracy: 100.00%
Time elapsed: 2.26 min
Epoch: 008/010 training accuracy: 98.44%
Time elapsed: 2.26 min
Epoch: 008/010 training accuracy: 98.44%
Time elapsed: 2.26 min
Epoch: 008/010 training accuracy: 100.00%
Time elapsed: 2.26 min
Epoch: 008/010 training accuracy: 100.00%
Time elapsed: 2.26 min
Epoch: 008/010 | Batch 350/938 | Cost: 0.0566
Epoch: 008/010 training accuracy: 98.44%
Time elapsed: 2.26 min
Epoch: 008/010 training accuracy: 98.44%
Time elapsed: 2.26 min
Epoch: 008/010 training accuracy: 100.00%
Time elapsed: 2.26 min
Epoch: 008/010 training accuracy: 100.00%
Time elapsed: 2.26 min
Epoch:

In [20]:
print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader)))

Test accuracy: 100.00%
